# Skills requested in Google job posts

### Introduction

Which **language, skills, and experience** should we add to our toolbox for getting a job in Google? Google publishes all of their jobs at `careers.google.com`. Niyamat Ullah scraped all of the job data from that site by capturing the source code of every job page with a tool called **Selenium**, extracting the job title, location, responsibilities, minimum and preferred qualifications.

### The data set

The file `google.csv` contains data from 1,250 job positions. The variables are:

* `company`: either Google or Youtube.

* `title`: the title of the job.

* `category`: the category of the job.

* `location`: the location of the job.

* `responsibilities`: the responsibilities for the job.

* `minqual`: the minimum qualifications for the job.

* `prefqual`: the preferred qualifications for the job.

Source: Kaggle.


### Importing the data

I import the data from a remote CSV fileas usual. Since the job posts do not come with an identifier, none of the columns of the source file can be used as the index. So, I don't use the argument `index_col` in this example. 

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/mcanela-iese/DataScience/main/Data/google.csv'
df = pd.read_csv(url)

As usual, I check, with the functions `info` and `head`, that the content of the file is as expected. A few missing values are detected in the last three columns.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   company           1250 non-null   object
 1   title             1250 non-null   object
 2   category          1250 non-null   object
 3   location          1250 non-null   object
 4   responsibilities  1235 non-null   object
 5   minqual           1236 non-null   object
 6   prefqual          1236 non-null   object
dtypes: object(7)
memory usage: 68.5+ KB


In [4]:
df.head()

,company,title,category,location,responsibilities,minqual,prefqual
0,Google,Google Cloud Program Manager,Program Management,Singapore,"Shape, shepherd, ship, and show technical prog...",BA/BS degree or equivalent practical experienc...,Experience in the business technology market a...
1,Google,"Supplier Development Engineer (SDE), Cable/Con...",Manufacturing & Supply Chain,"Shanghai, China",Drive cross-functional activities in the suppl...,BS degree in an Engineering discipline or equi...,"BSEE, BSME or BSIE degree.\nExperience of usin..."
2,Google,"Data Analyst, Product and Tools Operations, Go...",Technical Solutions,"New York, NY, United States",Collect and analyze data to draw insight and i...,"Bachelor’s degree in Business, Economics, Stat...",Experience partnering or consulting cross-func...
3,Google,"Developer Advocate, Partner Engineering",Developer Relations,"Mountain View, CA, United States","Work one-on-one with the top Android, iOS, and...",BA/BS degree in Computer Science or equivalent...,"Experience as a software developer, architect,..."
4,Google,"Program Manager, Audio Visual (AV) Deployments",Program Management,"Sunnyvale, CA, United States",Plan requirements with internal customers.\nPr...,BA/BS degree or equivalent practical experienc...,CTS Certification.\nExperience in the construc...


### Duplicated job posts

The number of duplicated rows in this data set is obtained in the usual way. `duplicated` returns a Boolean mask indicating the duplicated rows. Then, `sum` returns the number of `True` values in that mask.   

In [5]:
df.duplicated().sum()

123

Since there is no ID for the job post, we cannot decide whether the duplication is really wrong. It may be that the data collector was careless in the web scraping process, but also that Google posted repeatedly the same job offer (or whatever). For this example, I take those duplicates as errors, so I drop them. This can be done with the function `drop_duplicates`:

In [6]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1127 entries, 0 to 1249
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   company           1127 non-null   object
 1   title             1127 non-null   object
 2   category          1127 non-null   object
 3   location          1127 non-null   object
 4   responsibilities  1112 non-null   object
 5   minqual           1113 non-null   object
 6   prefqual          1113 non-null   object
dtypes: object(7)
memory usage: 70.4+ KB


### Exploring the company

The first column has the company name. The analysis is very simple, since the company is either Google or YouTube (a few cases). The function `value_counts` does the job.

In [7]:
df['company'].value_counts()

Google     1107
YouTube      20
Name: company, dtype: int64

### Exploring the job title

The data set seems to include many different job titles. `value_counts` returns one count for each title, so the number of unique job titles can be obtained as:

In [8]:
len(df['title'].value_counts())

794

*Note*. `len(df['title'].unique())` would give us the same number.

The top-10 titles are:

In [9]:
df['title'].value_counts().head(10)

Business Intern 2018                            33
Field Sales Representative, Google Cloud        17
Interaction Designer                            12
MBA Intern, Summer 2018                         10
User Experience Researcher                       9
MBA Intern 2018                                  7
BOLD Intern, Summer 2018                         7
User Experience Design Intern, Summer 2018       7
Partner Sales Engineer, Google Cloud             7
User Experience Research Intern, Summer 2018     6
Name: title, dtype: int64

The interns seem to dominate the picture, but, with 794 different titles, this quick view could be misleading. So, I check other possibilities. The function `str.contains` returns a Boolean mask indicating whether a given pattern is contained in every term of the series.

In [10]:
mask = df['title'].str.contains('Intern')
mask.value_counts()

False    1019
True      108
Name: title, dtype: int64

Let me try other expressions, to compare with 'Intern':

In [11]:
df['title'].str.contains('Sales').sum()

133

In [12]:
df['title'].str.contains('Cloud').sum()

266

In [13]:
df['title'].str.contains('Google Cloud').sum()

248

### Exploring the category

The job categories look more promising to the analyst, since they are described in a more systematic way. There are 23 different categories here:

In [14]:
len(df['category'].value_counts())

23

The top-10 categories are:

In [15]:
df['category'].value_counts().head(10)

Sales & Account Management      167
Marketing & Communications      154
Technical Solutions             101
Finance                          86
People Operations                85
User Experience & Design         84
Program Management               69
Business Strategy                65
Partnerships                     54
Legal & Government Relations     45
Name: category, dtype: int64

### Exploring the location

It is typical of these data sets that the location comes as *city, state, country* for US and Canada (sometimes also for India and others), but as *city, country* for other countries. The country can be extracted by deleting every sequence of characters that ends with a comma followed by a white space. A **regular expression** for this pattern is `'.+, '`. The dot (`.`) is a **wildcard** which stands for any character, and the plus sign (`+`) is a **quantifier** which is read as *any length*.

The method `str.replace` using the **empty string** as the replacement performs the deletion:

In [16]:
country = df['location'].str.replace(pat='.+, ', repl='', regex=True)
len(country.value_counts())

49

The argument `regex=True` may not be needed, depending of the version of Pandas that you use., but it is better to control these details, so the code gets more robust. Again, the same job can be done with `apply` and a lambda function, or with a `for` loop (using on the function `sub`, from the package `re`). 

There are 49 countries, though, as you can see below, most of the job requests are for US.

In [17]:
country.value_counts().head(10)

United States     572
Ireland            87
United Kingdom     58
Germany            53
Singapore          36
Australia          33
China              32
Japan              30
Taiwan             27
India              25
Name: location, dtype: int64

### Exploring the last three columns

There are missing values in the last three columns this data set. Let us see whether the missingness occurs for the same job posts. For each column, the method `isna`creates a Boolean mask. Combining the three masks with the OR operator (`|`):

In [18]:
df[df['responsibilities'].isna() | df['minqual'].isna() | df['prefqual'].isna()]

,company,title,category,location,responsibilities,minqual,prefqual
15,Google,Manufacturing Test Engineer,Hardware Engineering,"South San Francisco, CA, United States",NaN,NaN,NaN
72,Google,"Software Engineer, Android Applications, Veril...",Software Engineering,"Cambridge, MA, United States",NaN,NaN,NaN
91,Google,"Analog / Mixed Signal IC Design Engineer, Veri...",Hardware Engineering,"South San Francisco, CA, United States",NaN,NaN,NaN
97,Google,"Program Manager, Behavioral Health, Verily Lif...",Program Management,"South San Francisco, CA, United States",NaN,NaN,NaN
98,Google,"Software Engineer, Android Applications, Veril...",Software Engineering,"South San Francisco, CA, United States",NaN,NaN,NaN
123,Google,"Manufacturing Engineer, Verily Life Sciences -...",Hardware Engineering,"South San Francisco, CA, United States",NaN,NaN,NaN
150,Google,"IC Test Engineer, Verily Life Sciences - South...",Hardware Engineering,"South San Francisco, CA, United States",NaN,NaN,NaN
160,Google,"Software Engineer, Verily Life Sciences - Sout...",Software Engineering,"South San Francisco, CA, United States",NaN,NaN,NaN
202,Google,"Firmware Engineer, Verily Life Sciences - Sout...",Software Engineering,"South San Francisco, CA, United States",NaN,NaN,NaN
206,Google,"Software Test Engineer, Mobile and Web Applica...",Software Engineering,"Mountain View, CA, United States",NaN,NaN,NaN


For the analysis of these last columns, it is probably better to leave aside the job posts with missing information. This is done with the method `dropna`.

In [19]:
df = df.dropna()

With text data, missingness can also come as the empty string, white space, question marks (`?`), dashes (`-`), etc. One way to detect this is to check the length of the entries for these columns.

In [20]:
df['responsibilities'].str.len().min()

30

In [21]:
df['minqual'].str.len().min()

48

In [22]:
df['prefqual'].str.len().min()

72

I will focus the analysis to the column `prefqual`, which seems to have more content. 

### Analysis of the preferred qualifications

In this last part of the example, the analysis focuses on the most frequent terms in the preferred qualifications posted by Google. There are different Python packages where you could find specific tools for that, but I skip them here.

I put first everything in **lowercase**, with the method `str.lower`: 

In [23]:
prefqual = df['prefqual'].str.lower()
prefqual[0]

'experience in the business technology market as a program manager in saas, cloud computing, and/or emerging technologies.\nsignificant cross-functional experience across engineering, sales, and marketing teams in cloud computing or related technical fields.\nproven successful program outcomes from idea to launch in multiple contexts throughout your career.\nability to manage the expectations, demands and priorities of multiple internal stakeholders based on overarching vision and success for global team health.\nability to work under pressure and possess flexibility with changing needs and direction in a rapidly-growing organization.\nstrong organization and communication skills.'

Next, I extract the words with the method `findall`. As the pattern for the extraction, I use `\w+'`, which stands for any uninterrupted sequence of characters that can be used to form a word. This includes numbers but leaves out numbers, **punctuation** and the **control character** '\n', which means new line, and is used to separate paragraphs. `findall` returns a list called a **bag of words**. 

In this case, I will work with lists all the time:

In [24]:
import re
bags = [re.findall('\w+', p) for p in prefqual]

For, instance, the first of these bags contains 94 terms:

In [25]:
len(bags[0])

94

In [26]:
bags[0][:10]

['experience',
 'in',
 'the',
 'business',
 'technology',
 'market',
 'as',
 'a',
 'program',
 'manager']

Next, I flatten `bags`, to have a single list of terms, instead of a list of lists:

In [27]:
terms = [t for b in bags for t in b]

In total, I get 76,374 terms.

In [28]:
len(terms)

76374

We can get the top frequent terms, with `value_counts`. Note that `terms` has to be converted to a Pandas series, in order to apply `value_coounts`:

In [29]:
pd.Series(terms).value_counts().head(10)

and           5800
to            2676
in            2214
with          2202
experience    2168
ability       1618
of            1581
a             1517
or            1345
skills        1224
dtype: int64

As it could be expected, we find on top terms that do not convey information, which is typical. There terms are called **stopwords**. I use here a list which comes in the file `stopwords.csv`, extracted from an Internet source. The file has one column, with no header, and one word in every row.

In [30]:
url = 'https://raw.githubusercontent.com/mcanela-iese/DataScience/main/Data/stopwords.csv'
stopwords = pd.read_csv(url, header=None, squeeze=True)

The source file comes without header. So, I use the argument `header=None` has been to stop `read_csv` taking the first stopword as the name of the column. With `squeeze=True`, I get a series instead of a data frame with one column:

In [31]:
stopwords.shape

(571,)

Now, I convert `stopwords` to a list with the function `tolist`:

In [32]:
stopwords = stopwords.tolist()

Next, I drop all the stopwords from `terms`:

In [33]:
terms = [t for t in terms if t not in stopwords]

In [34]:
len(terms)

52060

The top-10 most frequent terms are, now:

In [35]:
pd.Series(terms).value_counts().head(10)

experience       2168
ability          1618
skills           1224
management        645
demonstrated      542
excellent         505
work              504
business          501
communication     501
strong            453
dtype: int64

The term on top is experience. What is the proportion of job posts mentioning experience in the preferred qualifications?

In [36]:
prefqual.str.contains('experience').mean().round(3)

0.853

Pretty high. Let me check a few terms more.

In [37]:
prefqual.str.contains('ability').mean().round(3)

0.758

In [38]:
prefqual.str.contains('skills').mean().round(3)

0.699

In [39]:
prefqual.str.contains('management').mean().round(3)

0.443

In [40]:
prefqual.str.contains('communication').mean().round(3)

0.476

In an alternative approach, you may start with a predefined list of words and check their inclusion in the preferred qualifications. This could be done without building the list of most frequent terms. For instance, suppose that you are interested in learning how often a languages like Python is explicitly mentioned:

In [41]:
prefqual.str.contains('python').mean().round(3)

0.062